## Trump Model Training

In [90]:
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.utils import to_categorical
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf


In [91]:
## define paths to data files
path_to_train_data = Path("C:\\Users\\matth\\Documents\\DL4G\\jass-data\\split\\train\\filtered\\card\\csv")
path_to_test_data = Path("C:\\Users\\matth\\Documents\\DL4G\\jass-data\\split\\test\\filtered\\card\\csv")
path_to_val_data = Path("C:\\Users\\matth\\Documents\\DL4G\\jass-data\\split\\val\\filtered\\card\\csv")

In [92]:
# train data
data_train1 = pd.read_csv(path_to_train_data / '0001.csv', header=None)
data_train2 = pd.read_csv(path_to_train_data / '0002.csv', header=None)
data_train3 = pd.read_csv(path_to_train_data / '0003.csv', header=None)
data_train4 = pd.read_csv(path_to_train_data / '0004.csv', header=None)
data_val1 = pd.read_csv(path_to_val_data / '0001.csv', header=None)
data_val2 = pd.read_csv(path_to_val_data / '0002.csv', header=None)

data_train = pd.concat([data_train1, data_train2, data_train3, data_train4, data_val1, data_val2], axis=0)
#data_val = pd.concat([data_val1, data_val2], axis=0)

In [93]:
# test data
data_test1 = pd.read_csv(path_to_test_data / '0001.csv', header=None)
data_test2 = pd.read_csv(path_to_train_data / '0002.csv', header=None)

data_test = pd.concat([data_test1, data_test2])
data_test.shape
data_test.head()

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,21
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,20
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,12
3,0,0,0,1,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,1,3
4,0,1,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,1,0,0,8


## Label data for readability

## Prepare x and y values

In [94]:
# colums used for extracting x and y values. The same effect could be achieved with train_test_split-Method, but 
# since we already have different files, we dont need to split the files using this method.
#data_X_columns = cards + forehand
#data_Y_colums = trump

x_train = data_train[data_train.columns[0:82]]
y_train = data_train[data_train.columns[82]]
#print (x_train.head())
#print (y_train.head())

x_test = data_test[data_test.columns[0:82]]
y_test = data_test[data_test.columns[82]]

## Create model and train it

In [95]:
# We have 37 features, so we have a node for each feature. There are 7 output categories: each trump color(4), 
# obe-abe, unde-ufe, schiebe. So we need an reducing function with 7 elements
model = keras.Sequential()
model.add(keras.layers.Dense(82, activation='relu', input_shape=[82]))
model.add(keras.layers.Dense(70, activation='relu'))
model.add(keras.layers.Dense(70, activation='relu'))
model.add(keras.layers.Dense(58, activation='relu'))
model.add(keras.layers.Dense(58, activation='relu'))
model.add(keras.layers.Dense(46, activation='relu'))
model.add(keras.layers.Dense(46, activation='relu'))
model.add(keras.layers.Dense(36, activation='relu'))
model.add(keras.layers.Dense(36, activation='softmax'))

opt = Adam()

model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

print(model.summary())


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 82)                6806      
_________________________________________________________________
dense_79 (Dense)             (None, 70)                5810      
_________________________________________________________________
dense_80 (Dense)             (None, 70)                4970      
_________________________________________________________________
dense_81 (Dense)             (None, 58)                4118      
_________________________________________________________________
dense_82 (Dense)             (None, 58)                3422      
_________________________________________________________________
dense_83 (Dense)             (None, 46)                2714      
_________________________________________________________________
dense_84 (Dense)             (None, 46)               

In [ ]:
y_train_categorical = to_categorical(y_train)

callbacks = [EarlyStopping(monitor='val_loss', patience=10),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

history = model.fit(x_train, y_train_categorical, validation_split=0.20, epochs=150, batch_size=128, callbacks=callbacks)

Train on 408132 samples, validate on 102033 samples
Epoch 1/150
408132/408132 [==============================] - 18s 43us/sample - loss: 2.1200 - accuracy: 0.3725 - val_loss: 1.6963 - val_accuracy: 0.4724
Epoch 2/150
408132/408132 [==============================] - 14s 35us/sample - loss: 1.5626 - accuracy: 0.5031 - val_loss: 1.4849 - val_accuracy: 0.5183
Epoch 3/150
408132/408132 [==============================] - 17s 40us/sample - loss: 1.4124 - accuracy: 0.5344 - val_loss: 1.3699 - val_accuracy: 0.5368
Epoch 4/150
380800/408132 [==========================>...] - ETA: 1s - loss: 1.3286 - accuracy: 0.5515

## Generate graphs for loss and accuracy

In [ ]:
plt.plot(history.history['loss'])
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['Train', 'Val'], loc='upper left')

## Test model

In [ ]:
y_test_categorical = to_categorical(y_test)
model.evaluate(x_test, y_test_categorical)

In [ ]:
model.save("card_model_e120_b256_sgd.h5")